In [6]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import os


# Suppress ROOT warnings for Sumw2
ROOT.gErrorIgnoreLevel = ROOT.kError

# ─── 0) 파일 경로 & 설정 ──────────────────────────────────────────────
data_files = [
#    "DoubleMuon_Data_2018.root",
#    "MuonEG_Data_2018.root",
    "EGamma_Data_2018.root",
    "SingleMuon_Data_2018.root",
]

mc_files = [
    "DYJetsToLL_M-10to50_MC_2018.root",
    "DYJetsToLL_M-50_MC_2018.root",
    "ST_s-channel_MC_2018.root",
    "ST_t-channel_antitop_MC_2018.root",
    "ST_t-channel_top_MC_2018.root",
    "ST_tW_antitop_MC_2018.root",
    "ST_tW_top_MC_2018.root",
    "TTTo2L2Nu_MC_2018.root",
    "TTToHadronic_MC_2018.root",
    "TTToSemiLeptonic_MC_2018.root",
    "WJetsToLNu_MC_2018.root",
    "WW_MC_2018.root",
    "WZ_MC_2018.root",
    "ZZ_MC_2018.root",
]

cross_sections_pb = {
    "DYJetsToLL_M-10to50_MC_2018.root": 18610.0,
    "DYJetsToLL_M-50_MC_2018.root":    6077.22,
    "ST_s-channel_MC_2018.root":       10.32,
    "ST_t-channel_antitop_MC_2018.root": 80.95,
    "ST_t-channel_top_MC_2018.root":   136.02,
    "ST_tW_antitop_MC_2018.root":      19.89,
    "ST_tW_top_MC_2018.root":          19.89,
    "TTTo2L2Nu_MC_2018.root":          89.05,
    "TTToHadronic_MC_2018.root":       831.76 * 0.45441,
    "TTToSemiLeptonic_MC_2018.root":   831.76 * 0.44113,
    "WJetsToLNu_MC_2018.root":         61526.0,
    "WW_MC_2018.root":                 118.7,
    "WZ_MC_2018.root":                 46.75,
    "ZZ_MC_2018.root":                 16.91,
}

luminosity_pb = 59.740 * 1000.0  # pb^-1
out_dir = "compare_plots_mpl_stack"
os.makedirs(out_dir, exist_ok=True)

# ─── 1) 히스토그램 이름 추출 ────────────────────────────────────────────
def get_hist_names(fn):
    f = ROOT.TFile.Open(fn)
    f.cd("plots")
    names = [k.GetName() for k in ROOT.gDirectory.GetListOfKeys()]
    f.Close()
    return names

hist_names = get_hist_names(mc_files[0])

# ─── 1.5) 이름 필터 적용 ────────────────────────────────────────────────
prefixes = ("mu", "e", "combine")
hist_names = [
    h for h in hist_names
    if any(h.startswith(f"{p}_") for p in prefixes)
]

# ─── 2) 루프: 각 히스토그램별 Data/MC 누적 & 스택 플롯 ────────────────────
for hname in hist_names:
    print("▶", hname)

    # --- 2-1) Data 누적 ---
    data_h = None
    for fn in data_files:
        f = ROOT.TFile.Open(fn)
        h = f.Get(f"plots/{hname}")
        if h:
            if data_h is None:
                data_h = h.Clone("data_tmp")
                data_h.SetDirectory(0)
            else:
                data_h.Add(h)
        f.Close()
    if data_h is None:
        print("   ✖ No data, skipping")
        continue

    # --- 2-2) MC 샘플별 스케일 & 리스트화 ---
    mc_counts_list = []
    mc_labels      = []
    for fn in mc_files:
        f = ROOT.TFile.Open(fn)
        h = f.Get(f"plots/{hname}")
        if not h:
            f.Close()
            continue

        cnt   = f.Get("plots/count")
        n_evt = cnt.GetSumOfWeights() if cnt else 0
        if n_evt <= 0:
            f.Close()
            continue

        xsec  = cross_sections_pb.get(fn, 1.0)
        scale = (xsec * luminosity_pb) / n_evt

        tmp = h.Clone("tmp"); tmp.SetDirectory(0); tmp.Sumw2()
        tmp.Scale(scale)

        nb  = tmp.GetNbinsX()
        arr = np.array([tmp.GetBinContent(i) for i in range(1, nb+1)])
        mc_counts_list.append(arr)
        mc_labels.append(os.path.splitext(fn)[0])
        f.Close()

    if not mc_counts_list:
        print("   ✖ No MC, skipping")
        continue

    # --- 2-3) 1D 히스토그램만 처리 ---
    if data_h.GetDimension() != 1:
        print("   ✖ Skipping non-1D histogram")
        continue

    # --- 2-4) NumPy 배열 변환 ---
    nb = data_h.GetNbinsX()
    edges = np.array(
        [data_h.GetBinLowEdge(i) for i in range(1, nb+1)]
        + [data_h.GetBinLowEdge(nb) + data_h.GetBinWidth(nb)]
    )
    dat_counts  = np.array([data_h.GetBinContent(i) for i in range(1, nb+1)])
    dat_err     = np.sqrt(dat_counts)
    bin_centers = 0.5 * (edges[:-1] + edges[1:])

    # --- 2-5) 그리기: CMS 스타일 2패널 (스택+ratio) ---
    plt.style.use(hep.style.CMS)
    fig, (ax, axr) = plt.subplots(
        nrows=2, sharex=True,
        gridspec_kw={"height_ratios":[3,1], "hspace":0.05},
        figsize=(10, 8), dpi=150
    )

    # MC 스택 플롯
    hep.histplot(
        mc_counts_list, bins=edges,
        stack=True, histtype="fill",
        label=mc_labels, ax=ax
    )
    # Data 오버레이
    ax.errorbar(
        bin_centers, dat_counts, yerr=dat_err,
        fmt="o", color="black", label="Data"
    )
    ax.set_ylabel("Events / bin")
    hep.cms.label("Private Work", data=True, lumi=59.8, year=2018, ax=ax)
    #ax.legend(loc="upper right", fontsize="small")
    ax.legend(
    loc="upper right",
    prop={"size":6},        # 폰트 크기
    handletextpad=0.2,      # 마커(색상 상자)와 텍스트 간격
    labelspacing=0.2,       # 레이블 간 세로 간격
    columnspacing=0.5,      # 칼럼 간 가로 간격 (ncol>1일 때)
)

    # Ratio 패널
    mc_sum = np.sum(mc_counts_list, axis=0)
    mask   = mc_sum > 0
    ratio     = np.divide(dat_counts, mc_sum,
                          out=np.zeros_like(dat_counts), where=mask)
    ratio_err = np.divide(dat_err,    mc_sum,
                          out=np.zeros_like(dat_err),    where=mask)
    axr.errorbar(
        bin_centers[mask], ratio[mask],
        yerr=ratio_err[mask], fmt="o", color="black"
    )
    axr.axhline(1, color="gray", linestyle="--", linewidth=1)
    axr.set_ylabel("Data/MC")
    axr.set_xlabel(hname)
    axr.set_ylim(0.5, 1.5)

    # 파일 저장
    fig.savefig(f"{out_dir}/stack_{hname}.png")
    plt.close(fig)

print("✅ All stacked plots saved in", out_dir)


▶ e_2_jet_1_btag_MET
▶ e_2_jet_1_btag_lep1pt
▶ e_2_jet_1_btag_lep1pt_None
▶ e_2_jet_1_btag_lep1eta
▶ e_2_jet_1_btag_jet1eta
▶ e_2_jet_1_btag_jet2eta
▶ e_2_jet_1_btag_jet3eta
▶ e_2_jet_1_btag_jet1pt
▶ e_2_jet_1_btag_jet2pt
▶ e_2_jet_1_btag_jet3pt
▶ mu_2_jet_1_btag_MET
▶ mu_2_jet_1_btag_lep1pt
▶ mu_2_jet_1_btag_lep1pt_None
▶ mu_2_jet_1_btag_lep1eta
▶ mu_2_jet_1_btag_jet1eta
▶ mu_2_jet_1_btag_jet2eta
▶ mu_2_jet_1_btag_jet3eta
▶ mu_2_jet_1_btag_jet1pt
▶ mu_2_jet_1_btag_jet2pt
▶ mu_2_jet_1_btag_jet3pt
▶ combine_2_jet_1_btag_MET
▶ combine_2_jet_1_btag_lep1pt
▶ combine_2_jet_1_btag_lep1pt_None
▶ combine_2_jet_1_btag_lep1eta
▶ combine_2_jet_1_btag_jet1eta
▶ combine_2_jet_1_btag_jet2eta
▶ combine_2_jet_1_btag_jet3eta
▶ combine_2_jet_1_btag_jet1pt
▶ combine_2_jet_1_btag_jet2pt
▶ combine_2_jet_1_btag_jet3pt
▶ e_2_jet_2_btag_MET
▶ e_2_jet_2_btag_lep1pt
▶ e_2_jet_2_btag_lep1pt_None
▶ e_2_jet_2_btag_lep1eta
▶ e_2_jet_2_btag_jet1eta
▶ e_2_jet_2_btag_jet2eta
▶ e_2_jet_2_btag_jet3eta
▶ e_2_jet_2_btag_j